# Class Generation

Generate classes for structures whose definitions were loaded with the utilities notebook.

1. [Test the Generation of a Class Definition](#test)
1. [Generate __all__](#all)
2. [generate_class Method Definition](#class)
3. [Run the Generation of All Classes](#run)


<a id="test"></a>
## Test the Generation of a Class Definition

In [3]:
out = generate_class("record-FAM")
print(out)


class RecordFam(BaseStructure):
    '''Store, validate and format the FAM structure. 
    Generated by the class_generation.ipynb notebook.

    GEDCOM Specification:
    > Family record
    > See FAMILY_RECORD
    > 
    > <div class="note">
    > 
    > The common case is that each couple has one FAM record, but that is not
    > always the case.
    > 
    > A couple that separates and then gets together again can be represented either
    > as a single FAM with multiple events (MARR, DIV, etc.) or as a separate
    > FAM for each time together. Some user interfaces may display these two in
    > different ways and the two admit different semantics in sourcing. A single
    > FAM with two MARR with distinct dates might also represent uncertainty
    > about dates and a pair of FAM with same spouses might also be the result of
    > merging multiple files.
    > 
    > Implementers should support both representations, and should choose between
    > them based on user input or other

<a id="all"></a>
## Generate __all__

In [ ]:
from genedata.gedcom7 import Structure
from genedata.constants import Default

print('__all__ = [')
for key in Structure:
    if key not in ["CONT"]:
        print(''.join(["    '", key.title().replace('_','').replace('-',''), "',"]))
print(']')

<a id="class"></a>
## generate_class Method Definition

In [2]:
from textwrap import wrap

from genedata.constants import Default
from genedata.gedcom7 import Enumeration, Examples, Structure


def get_class_name(key) -> str:
    return key.title().replace("_", "").replace("-", "")


def generate_specification(key: str) -> str:
    """Construct the Specification section of the documentation."""
    specification: list[str] = Structure[key][Default.YAML_SPECIFICATION]
    spec: str = """
    GEDCOM Specification:"""
    for string in specification:
        if "\n" not in string and len(string) > 80:
            wrapped = wrap(string, 75)
            if len(wrapped) > 1:
                for line in wrapped:
                    spec = "".join([spec, "\n    > ", line])
        else:
            spec = "".join(
                [spec, "\n    > ", string.replace("\n", "\n    > ").replace("`", "")]
            )
    return spec


def generate_examples(key: str) -> str:
    """Construct the Example section from the Examples dictionary in gedcom7."""
    class_name: str = get_class_name(key)
    tag = Structure[key][Default.YAML_STANDARD_TAG]
    example: str = Default.EMPTY
    if key in Examples:
        example = Examples[key]
    return example


def generate_enumerations(key: str) -> str:
    """Construct the Enumerations section of the documentation."""
    enumeration_set: str = Default.EMPTY
    if Default.YAML_ENUMERATION_SET in Structure[key]:
        enumeration_set = """
    
    Enumerations:"""
        for keyname, value in Enumeration.items():
            if (
                Structure[key][Default.YAML_ENUMERATION_SET]
                in value[Default.YAML_VALUE_OF]
            ):
                wrapped = wrap(value[Default.YAML_SPECIFICATION][0], 75)
                enumvalue: str = Default.EMPTY
                if len(wrapped) > 1:
                    for line in wrapped:
                        enumvalue = "".join([enumvalue, "\n        > ", line])
                else:
                    enumvalue = "".join(
                        [enumvalue, "\n        > ", wrapped[0].replace("\n", "\n    ")]
                    )
                enumeration_set = "".join(
                    [
                        enumeration_set,
                        "\n    - '",
                        value[Default.YAML_STANDARD_TAG],
                        "': ",
                        value[Default.YAML_URI],
                        enumvalue,
                    ]
                )
    return enumeration_set


def generate_substructures(key: str) -> str:
    """Construct the Substructures section of the documentation."""
    subs: list[str] = []
    if Default.YAML_SUBSTRUCTURES in Structure[key]:
        subs = Structure[key][Default.YAML_SUBSTRUCTURES]
    substructures: str = Default.EMPTY
    if len(subs) > 0:
        substructures = """
    
    Substructures:
    |               Specification                | Quantity | Required |
    | ------------------------------------------ | -------- | -------- |"""
        for key, value in subs.items():
            yes: str = "No"
            one: str = "Many"
            if Default.YAML_CARDINALITY_REQUIRED in value:
                yes = "Yes"
            if Default.YAML_CARDINALITY_SINGULAR in value:
                one = "Only One"
            substructures = "".join(
                [
                    substructures,
                    "\n    | ",
                    key,
                    " " * (42 - len(key)),
                    " | ",
                    one,
                    " " * (8 - len(one)),
                    " | ",
                    yes,
                    " " * (8 - len(yes)),
                    " |",
                ]
            )
    return substructures


def generate_args(key: str) -> str:
    """Construct the Args section of the documentation."""
    args = """
    
    Args:"""
    tag: str = Structure[key][Default.YAML_STANDARD_TAG]
    permitted: list[str] = Structure[key][Default.YAML_PERMITTED]
    if (
        Default.YAML_PAYLOAD in Structure[key]
        and Structure[key][Default.YAML_PAYLOAD] is not None
    ):
        args = "".join(
            [
                args,
                "\n        ",
                Default.CODE_VALUE,
                ": A value of data type ",
                Structure[key][Default.YAML_PAYLOAD],
            ]
        )
    args = "".join(
        [
            args,
            "\n        ",
            Default.CODE_SUBS,
            ": A permitted substructure, an extension or list of permitted substructures or extensions.",
        ]
    )
    return args


def generate_references(key: str) -> str:
    """Construct the References section of the documentation."""
    uri = Structure[key][Default.YAML_URI]
    tag = Structure[key][Default.YAML_STANDARD_TAG]
    return f"""
    
    References:
    - [GEDCOM {tag} Structure]({uri})"""


def get_datatype(key: str) -> str:
    """Convert the GEDCOM datatype into python datatype."""
    datatype: str = Default.EMPTY
    if Default.YAML_PAYLOAD in Structure[key] and Structure[key] is not None:
        datatype = Structure[key][Default.YAML_PAYLOAD]
    match datatype:
        case "http://www.w3.org/2001/XMLSchema#nonNegativeInteger":
            return "int"
        case "@<https://gedcom.io/terms/v7/record-INDI>@":
            return "IndividualXref"
        case "@<https://gedcom.io/terms/v7/record-FAM>@":
            return "FamilyXref"
        case "@<https://gedcom.io/terms/v7/record-SUBM>@":
            return "SubmitterXref"
        case "@<https://gedcom.io/terms/v7/record-OBJE>@":
            return "MultimediaXref"
        case "@<https://gedcom.io/terms/v7/record-REPO>@":
            return "RepositoryXref"
        case "@<https://gedcom.io/terms/v7/record-SNOTE>@":
            return "SharedNoteXref"
        case "@<https://gedcom.io/terms/v7/record-SOUR>@":
            return "SourceXref"
        case _:
            return "str"
    return "str"


def get_record_datatype(key: str) -> str:
    """Assign a cross reference datatype to specific records."""
    datatype: str = key[7:]
    match datatype:
        case "FAM":
            return "FamilyXref"
        case "INDI":
            return "IndividualXref"
        case "OBJE":
            return "MultimediaXref"
        case "REPO":
            return "RepositoryXref"
        case "SNOTE":
            return "SharedNoteXref"
        case "SOUR":
            return "SourceXref"
        case "SUBM":
            return "SubmitterXref"
        case _:
            return "Xref"
    return "Xref"


def generate_init(key: str) -> str:
    """Construct the global constants and init section of the class."""
    permitted: list[str] = Structure[key][Default.YAML_PERMITTED]
    required: list[str] = Structure[key][Default.YAML_REQUIRED]
    value_arg: str = f', {Default.CODE_VALUE}: {get_datatype(key)}'
    value_init: str = Default.CODE_VALUE
    subs_arg: str = f', {Default.CODE_SUBS}: Any'
    if len(required) == 0:
        subs_arg: str = f', {Default.CODE_SUBS}: Any = None'
    subs_init: str = Default.CODE_SUBS
    payload: str = Default.EMPTY
    if (
        Default.YAML_PAYLOAD in Structure[key]
        and Structure[key][Default.YAML_PAYLOAD] is not None
    ):
        payload = Structure[key][Default.YAML_PAYLOAD]
    init: str = f"\n    key: str = '{key}'"
    if payload == Default.EMPTY:
        value_arg = Default.EMPTY
        value_init = "Default.EMPTY"
        if "record-" in key:
            init = f"\n    key: str = '{key}'"
            value_arg = f', {Default.CODE_VALUE}: {get_record_datatype(key)}'
            value_init = Default.CODE_VALUE
    init_line: str = f"""
    
    def __init__(self{value_arg}{subs_arg}) -> None:
        super().__init__({value_init}, {subs_init}, self.key)"""
    return "".join([init, init_line])


def generate_class(key: str) -> str:
    """Construct the class documentation."""
    tag: str = Structure[key][Default.YAML_STANDARD_TAG]
    class_name: str = get_class_name(key)
    lines: str = f"""
class {class_name}(BaseStructure):
    '''Store, validate and format the {tag} structure. 
    Generated by the class_generation.ipynb notebook.
{generate_specification(key)}{generate_examples(key)}{generate_enumerations(key)}{generate_substructures(key)}{generate_args(key)}{generate_references(key)}
    '''
{generate_init(key)}
"""
    return lines

<a id="run"></a>
## Run the Generation of All Classes

In [4]:
from genedata.gedcom7 import Structure

for key, value in Structure.items():
    if key not in ["CONT"]:
        print(generate_class(key))


class Abbr(BaseStructure):
    '''Store, validate and format the ABBR structure. 
    Generated by the class_generation.ipynb notebook.

    GEDCOM Specification:
    > Abbreviation
    > A short name of a title, description, or name used for sorting, filing, and
    > retrieving records.
    
    Args:
        value: A value of data type http://www.w3.org/2001/XMLSchema#string
        subs: A permitted substructure, an extension or list of permitted substructures or extensions.
    
    References:
    - [GEDCOM ABBR Structure](https://gedcom.io/terms/v7/ABBR)
    '''

    key: str = 'ABBR'
    
    def __init__(self, value: str, subs: Any = None) -> None:
        super().__init__(value, subs, self.key)


class Addr(BaseStructure):
    '''Store, validate and format the ADDR structure. 
    Generated by the class_generation.ipynb notebook.

    GEDCOM Specification:
    > Address
    > The location of, or most relevant to, the subject of the superstructure.
    > See `ADDRESS_STRUCTUR